In [8]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv
from sklearn.ensemble import RandomForestClassifier

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({2:1},inplace=True)

In [7]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [4]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 4.9 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.833523,0.931944,0.887755,0.882734,0.931944,0.833523,0.887755,0.882734,0.931944,0.833523,0.887755,0.882734
std,0.032619,0.023516,0.016553,0.017168,0.023516,0.032619,0.016553,0.017168,0.023516,0.032619,0.016553,0.017168
min,0.784091,0.879630,0.867347,0.859638,0.879630,0.784091,0.867347,0.859638,0.879630,0.784091,0.867347,0.859638
25%,0.803977,0.914352,0.872449,0.869003,0.914352,0.803977,0.872449,0.869003,0.914352,0.803977,0.872449,0.869003
50%,0.829545,0.935185,0.885204,0.880051,0.935185,0.829545,0.885204,0.880051,0.935185,0.829545,0.885204,0.880051
75%,0.866477,0.953704,0.895408,0.892729,0.953704,0.866477,0.895408,0.892729,0.953704,0.866477,0.895408,0.892729
max,0.875000,0.953704,0.918367,0.914352,0.953704,0.875000,0.918367,0.914352,0.953704,0.875000,0.918367,0.914352


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  8.6min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=400, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [7]:
rf_search_rand.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 0.7,
 'max_depth': None,
 'bootstrap': False}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.893939,0.924897,0.910998,0.909418,0.924897,0.893939,0.910998,0.909418,0.924897,0.893939,0.910998,0.909418
std,0.021259,0.034160,0.023810,0.022967,0.034160,0.021259,0.023810,0.022967,0.034160,0.021259,0.023810,0.022967
min,0.875000,0.879630,0.877551,0.877315,0.879630,0.875000,0.877551,0.877315,0.879630,0.875000,0.877551,0.877315
25%,0.875000,0.879630,0.887755,0.888678,0.879630,0.875000,0.887755,0.888678,0.879630,0.875000,0.887755,0.888678
50%,0.886364,0.944444,0.913265,0.909722,0.944444,0.886364,0.913265,0.909722,0.944444,0.886364,0.913265,0.909722
75%,0.920455,0.944444,0.933673,0.932449,0.944444,0.920455,0.933673,0.932449,0.944444,0.920455,0.933673,0.932449
max,0.920455,0.953704,0.938776,0.937079,0.953704,0.920455,0.938776,0.937079,0.953704,0.920455,0.938776,0.937079


## Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [1100, 1200, 1300],
               'max_features': [.6, .7, .8],
               'max_depth': [None, 100, 120],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed: 13.5min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(bootstrap=False, max_depth=70,
                                              max_features=0.2,
                                              n_estimators=1500,
                                              random_state=420),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True],
                         'max_depth': [None, 100, 120],
                         'max_features': [0.6, 0.7, 0.8],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [4, 5, 6],
                         'n_estimators': [1100, 1200, 1300]},
             verbose=2)

In [12]:
rf_search_grid.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 0.6,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 1100}

In [13]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.871940,0.934367,0.906179,0.903153,0.934367,0.871940,0.906179,0.903153,0.934367,0.871940,0.906179,0.903153
std,0.006919,0.013845,0.005527,0.004761,0.013845,0.006919,0.005527,0.004761,0.013845,0.006919,0.005527,0.004761
min,0.864407,0.911628,0.895408,0.893667,0.911628,0.864407,0.895408,0.893667,0.911628,0.864407,0.895408,0.893667
25%,0.864407,0.920930,0.903061,0.901143,0.920930,0.864407,0.903061,0.901143,0.920930,0.864407,0.903061,0.901143
50%,0.870056,0.944186,0.908163,0.904296,0.944186,0.870056,0.908163,0.904296,0.944186,0.870056,0.908163,0.904296
75%,0.875706,0.944186,0.908163,0.904796,0.944186,0.875706,0.908163,0.904796,0.944186,0.875706,0.908163,0.904796
max,0.881356,0.944186,0.913265,0.909946,0.944186,0.881356,0.913265,0.909946,0.944186,0.881356,0.913265,0.909946


In [19]:
rf = RandomForestClassifier(n_estimators=1100, max_features=.6, max_depth = 80, min_samples_split = 4, min_samples_leaf = 4, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 1min 49s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.848023,0.926977,0.891327,0.887500,0.926977,0.848023,0.891327,0.887500,0.926977,0.848023,0.891327,0.887500
std,0.024242,0.026956,0.009268,0.008584,0.026956,0.024242,0.009268,0.008584,0.026956,0.024242,0.009268,0.008584
min,0.824859,0.888372,0.880102,0.875719,0.888372,0.824859,0.880102,0.875719,0.888372,0.824859,0.880102,0.875719
25%,0.830508,0.915116,0.886480,0.881908,0.915116,0.830508,0.886480,0.881908,0.915116,0.830508,0.886480,0.881908
50%,0.836158,0.927907,0.890306,0.886855,0.927907,0.836158,0.890306,0.886855,0.927907,0.836158,0.890306,0.886855
75%,0.853107,0.938372,0.896684,0.894160,0.938372,0.853107,0.896684,0.894160,0.938372,0.853107,0.896684,0.894160
max,0.892655,0.967442,0.908163,0.901800,0.967442,0.892655,0.908163,0.901800,0.967442,0.892655,0.908163,0.901800


In [10]:
# All samples
rf = RandomForestClassifier(n_estimators=1100, max_features=.6, max_depth = 80, min_samples_split = 4, min_samples_leaf = 4, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 41min 18s


,Sensitivity0,Specificity0,PPV0,NPV0,Accuracy0,Sensitivity1,Specificity1,PPV1,NPV1,Accuracy1,Sensitivity,Specificity,PPV,NPV,F1_Score,Accuracy
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.852558,0.939788,0.917607,0.891449,0.901514,0.939788,0.852558,0.891449,0.917607,0.901514,0.939788,0.852558,0.891449,0.917607,0.900956,0.901514
std,0.036722,0.015062,0.018462,0.023815,0.016154,0.015062,0.036722,0.023815,0.018462,0.016154,0.015062,0.036722,0.023815,0.018462,0.016453,0.016154
min,0.790698,0.909091,0.878049,0.854054,0.874150,0.909091,0.790698,0.854054,0.878049,0.874150,0.909091,0.790698,0.854054,0.878049,0.873546,0.874150
25%,0.821705,0.927273,0.904762,0.868571,0.887755,0.927273,0.821705,0.868571,0.904762,0.887755,0.927273,0.821705,0.868571,0.904762,0.887356,0.887755
50%,0.852713,0.939394,0.921260,0.892756,0.901361,0.939394,0.852713,0.892756,0.921260,0.901361,0.939394,0.852713,0.892756,0.921260,0.900887,0.901361
75%,0.883721,0.951515,0.933607,0.912281,0.914966,0.951515,0.883721,0.912281,0.933607,0.914966,0.951515,0.883721,0.912281,0.933607,0.914760,0.914966
max,0.914729,0.969697,0.956522,0.934524,0.935374,0.969697,0.914729,0.934524,0.956522,0.935374,0.969697,0.914729,0.934524,0.956522,0.935285,0.935374


# Outputs

In [11]:
### Save outputs ###
q = 'CDR_Imp_AllFeatures_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)